In [ ]:
import numpy as np
import pandas as pd
import itertools
import os
os.chdir("/content/drive/MyDrive/projects/Store_Sales-Time_Series_Forecasting/")
from models.model import LinearRegression_log, Ridge_log
from sklearn.metrics import mean_squared_log_error

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

print(mean_absolute_percentage_error([2], [52]))
print(mean_absolute_percentage_error([70], [122]))

25.0
0.7428571428571429


In [ ]:
# 特徴量
df_add_f = pd.read_csv("store-sales-time-series-forecasting/train_add_f.csv", index_col=0, parse_dates=True)
df_add_f.head()

,dcoilwtico,avg_oil,dayofweek_cos,dayofweek_sin,wd,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day,wageday,school_season
2013-01-01,NaN,NaN,-0.222521,0.974928,0,0,0,0,1,0,0,0,0
2013-01-02,93.140000,NaN,-0.900969,0.433884,1,0,0,0,0,0,0,0,0
2013-01-03,92.970000,NaN,-0.900969,-0.433884,1,0,0,0,0,0,0,0,0
2013-01-04,93.120000,NaN,-0.222521,-0.974928,1,0,0,0,0,0,0,0,0
2013-01-05,93.146667,NaN,0.623490,-0.781831,1,0,0,0,0,0,1,0,0


In [ ]:
# trainデータ
df_train = pd.read_csv("store-sales-time-series-forecasting/train.csv", index_col=1, parse_dates=True)
df_train.head()

,id,store_nbr,family,sales,onpromotion
date,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0
2013-01-01,1,1,BABY CARE,0.0,0
2013-01-01,2,1,BEAUTY,0.0,0
2013-01-01,3,1,BEVERAGES,0.0,0
2013-01-01,4,1,BOOKS,0.0,0


In [ ]:
df_test = pd.read_csv("store-sales-time-series-forecasting/test.csv", index_col=1, parse_dates=True)
df_test.head()

,id,store_nbr,family,onpromotion
date,,,,
2017-08-16,3000888,1,AUTOMOTIVE,0
2017-08-16,3000889,1,BABY CARE,0
2017-08-16,3000890,1,BEAUTY,2
2017-08-16,3000891,1,BEVERAGES,20
2017-08-16,3000892,1,BOOKS,0


In [ ]:
alphas = [0.01, 0.1, 1.0, 10]
result = []
family = 'SCHOOL AND OFFICE SUPPLIES'

for nbr in np.sort(df_train["store_nbr"].unique()):
  rmsles = []
  for alpha in alphas:
    df_y = df_train.query(" (family==@family) & (store_nbr==@nbr) ")
    
    y_train = df_y.loc["2016-08-01" : "2016-08-31"]["sales"]
    X_train = df_add_f.loc["2016-08-01"  : "2016-08-31"].drop(["avg_oil"], axis=1)
    y_test = df_y.loc["2017-08-01":]["sales"]
    X_test = df_add_f.loc["2017-08-01":"2017-08-15"].drop(["avg_oil"], axis=1)
    
    model = Ridge_log(alpha=alpha)

    model.fit(X_train, y_train)
    y_apply = model.predict(X_train)
    y_pred = model.predict(X_test)
    y_pred[y_pred<0] = 1

    rmsle = mean_squared_log_error(y_test, y_pred, squared=False)
    rmsles.append(rmsle)

  best_params = pd.DataFrame({"all_params" : alphas, "rmsles" : rmsles}).sort_values("rmsles").iloc[0].to_list()
  best_params.extend([family, nbr])
  result.append(best_params)

In [ ]:
df_result = pd.DataFrame(result)
df_result.columns = ["params", "val_RMSLE", "family", "store_nbr"]
df_result = df_result.reset_index(drop=True)
print(df_result["val_RMSLE"].mean())
df_result.head()

0.867372466702897


,params,val_RMSLE,family,store_nbr
0,1.00,0.000000,SCHOOL AND OFFICE SUPPLIES,1
1,0.01,0.400189,SCHOOL AND OFFICE SUPPLIES,2
2,10.00,0.933372,SCHOOL AND OFFICE SUPPLIES,3
3,0.01,0.420183,SCHOOL AND OFFICE SUPPLIES,4
4,0.01,0.727779,SCHOOL AND OFFICE SUPPLIES,5


In [ ]:
# 提出用のcsvを作るために、idと(date, store_nbr, family)を紐づける辞書を作る
dates = df_test.index.tolist()
familys = df_test["family"].tolist()
store_nbrs = df_test["store_nbr"].tolist()
ids = df_test["id"].tolist()
id_dic = dict()

for date, nbr, family, id in zip(dates, store_nbrs, familys, ids):
  if (date, nbr, family) not in id_dic:
    id_dic[(date, nbr, family)] = id

In [ ]:
familys = df_result["family"].unique().tolist()
store_nbrs = df_result["store_nbr"].unique().tolist()
a = list(itertools.product(store_nbrs, familys))
params = df_result["params"].tolist()

df_sub = pd.DataFrame()

for (nbr, family), param in zip(a, params):
  df_y = df_train.query(" (family==@family) & (store_nbr==@nbr) ")
    
  y_train = df_y.loc["2016-08-01" : "2016-08-31"]["sales"]
  X_train = df_add_f.loc["2016-08-01"  : "2016-08-31"].drop(["avg_oil"], axis=1)
  X_sub = df_add_f.loc["2017-08-16":"2017-08-31"].drop(["avg_oil"], axis=1)
  model = Ridge_log(alpha=param)
  model.fit(X_train, y_train)
  y_sub = model.predict(X_sub)
  y_sub[y_sub<0] = 1

  # date, nbr, familyでsubmission用のidを取り出す
  ids = [id_dic[(date, nbr, family)] for date in X_sub.index.tolist()]
  _sub = pd.DataFrame({"id" : ids, "sales" : y_sub})
  df_sub= pd.concat([df_sub, _sub])
df_sub = df_sub.sort_values(["id"])

In [ ]:
df_sub.head()

,id,sales
0,3000919,0.0
0,3000952,7.0
0,3000985,9.0
0,3001018,2.0
0,3001051,1.0


In [ ]:
# 上書き
df_result = pd.read_csv("submission/2209191331_Ridge+LightGBM.csv")
super_id = df_sub["id"].tolist()

ids = df_result.id.tolist()
old_sales = df_result.sales.tolist()
new_result = []
for id, sale in zip(ids, old_sales):
  if id not in super_id:
    new_result.append([id, sale])

for id, sale in zip(super_id, df_sub["sales"].tolist()):
  new_result.append([id, sale])

new_result = pd.DataFrame(new_result, columns=["id", "sales"]).sort_values(["id"])
new_result.head()

,id,sales
0,3000888,4.0
1,3000889,0.0
2,3000890,4.0
3,3000891,2100.0
4,3000892,0.0


In [ ]:
from datetime import datetime

model_name = "Ridge_log_selctedSchoolTrainPeriod+LightGBM"
filename = datetime.now().strftime(format="%y%m%d%H%M") + '_' + model_name
new_result.to_csv(f"submission/{filename}.csv", index=None)